# import libraries

In [ ]:
import os
import math, random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import init
from torchaudio import transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import torchaudio

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

# Loading Dataset From kaggle

In [ ]:
!pip install kaggle
!pip install torchsummary

In [ ]:
from google.colab import files

# Upload the kaggle.json file
files.upload()

# Move kaggle.json to the .kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k

# Extract urbansound8k voices

In [ ]:
import zipfile

zip_file_path = 'urbansound8k.zip'
extract_to_path = 'urbansound8k'

os.makedirs(extract_to_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Contents of {zip_file_path} extracted to {extract_to_path}")

# Torch Seed initialization

In [11]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(seed=2023)

# Read initial Data


In [12]:
meta_df = pd.read_csv("urbansound8k/UrbanSound8K.csv")
meta_df['relative_path'] = '/fold' + meta_df['fold'].astype(str) + '/' + meta_df['slice_file_name'].astype(str)
meta_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,relative_path
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,/fold5/100263-2-0-137.wav


# Take relevant columns

In [13]:
df = meta_df[['relative_path', 'classID']]
df.head()

,relative_path,classID
0,/fold5/100032-3-0-0.wav,3
1,/fold5/100263-2-0-117.wav,2
2,/fold5/100263-2-0-121.wav,2
3,/fold5/100263-2-0-126.wav,2
4,/fold5/100263-2-0-137.wav,2


# Voice preprocessing Functions

In [14]:
class AudioUtil:

    @staticmethod
    def open(audio_file): # Load an audio file. Return the signal as a tensor and the sample rate.
        sig, sr = torchaudio.load(audio_file)
        return sig, sr


    @staticmethod
    def rechannel(aud, new_channel): # Convert the given audio to the desired number of channels
        sig, sr = aud  #sig -> signal, sr -> sampling rate

        if sig.shape[0] == new_channel:
            return aud

        if new_channel == 1:
            resig = sig[:1, :] # Convert from stereo to mono by selecting only the first channel

        else:
            resig = torch.cat([sig, sig]) # Convert from mono to stereo by duplicating the first channel

        return resig, sr


    @staticmethod
    def resample(aud, newsr): # Since Resample applies to a single channel, we resample one channel at a time
        sig, sr = aud

        if sr == newsr:
            return aud

        num_channels = sig.shape[0]
        # Resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
        if num_channels > 1:
            # Resample the second channel and merge both channels
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
            resig = torch.cat([resig, retwo])

        return resig, newsr


    @staticmethod
    def pad_trunc(aud, max_ms): # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms

        if sig_len > max_len:
            # Truncate the signal to the given length
            sig = sig[:,:max_len]

        elif sig_len < max_len:
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            sig = torch.cat((pad_begin, sig, pad_end), 1)

        return sig, sr



    @staticmethod
    def time_shift(aud, shift_limit):     # Shifts the signal to the left or right by some percent. Values at the end
                                          # are 'wrapped around' to the start of the transformed signal.
        sig,sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return sig.roll(shift_amt), sr


    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None): # Generate a Spectrogram
        sig,sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return spec


    # ----------------------------
    # Augment the Spectrogram by masking out some sections of it in both the frequency
    # dimension (ie horizontal bars) and the time dimension (vertical bars) to prevent
    # overfitting and to help the model generalise better. The masked sections are
    # replaced with the mean value.
    # ----------------------------
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec

# implementation of dataloader

In [15]:
class SoundDS(Dataset):
    def __init__(self, df, data_path):
        self.df = df
        self.data_path = str(data_path)
        self.duration = 4000
        self.sr = 44100 #  sampling rate
        self.channel = 2
        self.shift_pct = 0.4

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_file = self.data_path + self.df.loc[idx, 'relative_path']
        class_id = self.df.loc[idx, 'classID']

        # Reading & Augmenting Data
        aud = AudioUtil.open(audio_file)
        reaud = AudioUtil.resample(aud, self.sr)
        rechan = AudioUtil.rechannel(reaud, self.channel)
        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)

        # Spectrogram
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

        return aug_sgram, class_id

In [16]:
data_path = "urbansound8k"
myds = SoundDS(df, data_path)

num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=128, shuffle=False)

In [17]:
next(iter(train_dl))[0].shape

torch.Size([32, 2, 64, 344])

# Model

In [18]:
class AudioClassifier (nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # The third Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # The 4th Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)


    def forward(self, x):
        x = self.conv(x)

        x = self.ap(x)

        x = x.view(x.shape[0], -1)
        x = self.lin(x)

        return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = AudioClassifier()
model = model.to(device)

next(model.parameters()).device

device(type='cuda', index=0)

# Train Model

In [19]:
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    for i, data in enumerate(train_dl):

        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s


        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        # if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch, i, running_loss / 10))


    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')

num_epochs=20
training(model, train_dl, num_epochs)

Epoch: 0, Loss: 2.11, Accuracy: 0.25
Epoch: 1, Loss: 1.86, Accuracy: 0.37
Epoch: 2, Loss: 1.66, Accuracy: 0.43
Epoch: 3, Loss: 1.49, Accuracy: 0.48
Epoch: 4, Loss: 1.36, Accuracy: 0.54
Epoch: 5, Loss: 1.23, Accuracy: 0.58
Epoch: 6, Loss: 1.11, Accuracy: 0.62
Epoch: 7, Loss: 1.02, Accuracy: 0.66
Epoch: 8, Loss: 0.95, Accuracy: 0.68
Epoch: 9, Loss: 0.89, Accuracy: 0.71
Epoch: 10, Loss: 0.83, Accuracy: 0.72
Epoch: 11, Loss: 0.80, Accuracy: 0.73
Epoch: 12, Loss: 0.77, Accuracy: 0.75
Epoch: 13, Loss: 0.74, Accuracy: 0.76
Epoch: 14, Loss: 0.72, Accuracy: 0.76
Epoch: 15, Loss: 0.69, Accuracy: 0.77
Epoch: 16, Loss: 0.67, Accuracy: 0.78
Epoch: 17, Loss: 0.65, Accuracy: 0.78
Epoch: 18, Loss: 0.66, Accuracy: 0.78
Epoch: 19, Loss: 0.64, Accuracy: 0.79
Finished Training


# Check model

In [22]:
import torchsummary
torchsummary.summary(model, (2, 64, 44))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 22]             408
            Conv2d-2            [-1, 8, 32, 22]             408
              ReLU-3            [-1, 8, 32, 22]               0
              ReLU-4            [-1, 8, 32, 22]               0
       BatchNorm2d-5            [-1, 8, 32, 22]              16
       BatchNorm2d-6            [-1, 8, 32, 22]              16
            Conv2d-7           [-1, 16, 16, 11]           1,168
            Conv2d-8           [-1, 16, 16, 11]           1,168
              ReLU-9           [-1, 16, 16, 11]               0
             ReLU-10           [-1, 16, 16, 11]               0
      BatchNorm2d-11           [-1, 16, 16, 11]              32
      BatchNorm2d-12           [-1, 16, 16, 11]              32
           Conv2d-13             [-1, 32, 8, 6]           4,640
           Conv2d-14             [-1, 3